In [1]:
import csv
import copy

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
dataset = './point_history.csv'
model_save_path = './gesture_classifier.hdf5'

In [3]:
TIME_STEPS = 16
DIMENSION = 2
NUM_CLASSES = 3

In [4]:
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (TIME_STEPS * DIMENSION) + 1 + 2)))

In [5]:
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))

In [6]:
for dataset_index, X_data in enumerate(X_dataset):
    for data_index in range(0, len(X_data), 2):
        if data_index == 0:
            image_width = X_data[data_index]
            image_height = X_data[data_index + 1]
            continue

        if data_index == 2:
            base_x = X_data[data_index]
            base_y = X_data[data_index + 1]

        X_data[data_index] = (X_data[data_index] - base_x) / image_width
        X_data[data_index + 1] = (X_data[data_index + 1] - base_y) / image_height

In [7]:
X_dataset = np.delete(X_dataset, [0, 1, 2, 3], 1)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=42)

In [9]:
input_size = (TIME_STEPS * DIMENSION) - 2

model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(input_size, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

2022-02-13 17:02:13.984272: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout (Dropout)           (None, 30)                0         
                                                                 
 dense (Dense)               (None, 24)                744       
                                                                 
 dropout_1 (Dropout)         (None, 24)                0         
                                                                 
 dense_1 (Dense)             (None, 10)                250       
                                                                 
 dense_2 (Dense)             (None, 3)                 33        
                                                                 
Total params: 1,027
Trainable params: 1,027
Non-trainable params: 0
_________________________________________________________________


In [11]:
# モデルコンパイル
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [12]:
# モデルチェックポイントのコールバック
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
# 早期打ち切り用コールバック
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [13]:
model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/1000
1/2 [==============>...............] - ETA: 0s - loss: 1.1473 - accuracy: 0.2109
Epoch 00001: saving model to ./gesture_classifier.hdf5
2/2 [==============================] - 1s 220ms/step - loss: 1.1294 - accuracy: 0.2626 - val_loss: 1.1148 - val_accuracy: 0.6515
Epoch 2/1000
1/2 [==============>...............] - ETA: 0s - loss: 1.1263 - accuracy: 0.3125
Epoch 00002: saving model to ./gesture_classifier.hdf5
2/2 [==============================] - 0s 81ms/step - loss: 1.1136 - accuracy: 0.3788 - val_loss: 1.1062 - val_accuracy: 0.9091
Epoch 3/1000
1/2 [==============>...............] - ETA: 0s - loss: 1.0876 - accuracy: 0.5547
Epoch 00003: saving model to ./gesture_classifier.hdf5
2/2 [==============================] - 0s 38ms/step - loss: 1.0960 - accuracy: 0.5556 - val_loss: 1.0977 - val_accuracy: 0.9091
Epoch 4/1000
1/2 [==============>...............] - ETA: 0s - loss: 1.1019 - accuracy: 0.7188
Epoch 00004: saving model to ./gesture_classifier.hdf5
2/2 [==============

In [14]:
# 推論テスト
predict_result = model.predict(np.array([X_test[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))

[9.9999952e-01 2.8274656e-07 2.4985874e-07]
0


In [15]:
%%timeit

# 時間計測
predict_result = model.predict(np.array([X_test[0]]))

32.2 ms ± 2.27 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
# 推論専用のモデルとして保存
model.save(model_save_path, include_optimizer=False)

In [17]:
model = tf.keras.models.load_model(model_save_path)

In [18]:
tflite_save_path = './gesture_classifier.tflite'

In [19]:
# モデルを変換(量子化)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

open(tflite_save_path, 'wb').write(tflite_quantized_model)

2022-02-13 17:02:58.087117: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /var/folders/_c/yfkk358d56n84q0rcrprvkc00000gn/T/tmpods81sw1/assets


2022-02-13 17:02:59.789030: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:363] Ignored output_format.
2022-02-13 17:02:59.789053: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:366] Ignored drop_control_dependency.
2022-02-13 17:02:59.793630: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /var/folders/_c/yfkk358d56n84q0rcrprvkc00000gn/T/tmpods81sw1
2022-02-13 17:02:59.797801: I tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2022-02-13 17:02:59.797827: I tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: /var/folders/_c/yfkk358d56n84q0rcrprvkc00000gn/T/tmpods81sw1
2022-02-13 17:02:59.804585: I tensorflow/cc/saved_model/loader.cc:210] Restoring SavedModel bundle.
2022-02-13 17:02:59.847877: I tensorflow/cc/saved_model/loader.cc:194] Running initialization op on SavedModel bundle at path: /var/folders/_c/yfkk358d56n84q0rcrprvkc00000gn/T/tmpods81sw1
2022-02

6064

In [20]:
interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()

In [21]:
# 入出力テンソルを取得
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [22]:
# 推論実施
interpreter.set_tensor(input_details[0]['index'], np.array([X_dataset[0]]))
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

print(np.squeeze(tflite_results))
print(np.argmax(np.squeeze(tflite_results)))

[9.9999774e-01 1.1452242e-06 1.0725973e-06]
0


In [23]:
%%timeit
# 推論実施
interpreter.set_tensor(input_details[0]['index'], np.array([X_dataset[0]]))
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

5.68 µs ± 250 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
